# Trabajo Práctico 1: Análisis Exploratorio de Datos

En este trabajo se va a utilizar un dataset de beisbol con el que más adelante se va a tratar de predecir si una jugada es home run o no. Pero para predecir lo anterior, primero se debe realizar el análisis de los datos del dataset, que es el objetivo de este TP.

## 1. Listado de variables y selección

### Detalle de variables

                            Nombre                                             Tipo

    - bip_id: identificador único de la pelota en juego                     No utilizada
    - game_date: fecha del juego. Formato (YYYY-MM-DD)                      Entrada (?)
    - home_team: abreviación del equipo local                               Entrada
    - away_team: abreviación del equipo visitante                           Entrada
    - batter_team: abreviación del equipo del bateador                      Entrada
    - batter_name: nombre del bateador                                      Entrada
    - pitcher_name: nombre del lanzador                                     Entrada
    - batter_id: identificador único del bateador                           No utilizada
    - pitcher_id: identificador único del lanzador                          No utilizada                             
    - is_batter_lefty: codificaciń binaria de bateadores zurdos             Entrada
    - is_pitcher_lefty: codificación binaria de lanzadores zurdos           Entrada
    - bb_type: clasificación del tipo de bola bateada                       Entrada
    - bearing: clasificación de la dirección horizontal de la pelota        Entrada (?)
    - pitch_name: nombre del tipo de lanzamiento                            Entrada
    - park: identificador único del estadio                                 No utilizada
    - inning: número de entradas dentro del juego                           Entrada
    - outs_when_up: número actual de outs                                   Entrada                          
    - balls: número actual de bolas                                         Entrada
    - strikes: número actual de strikes                                     Entrada
    - plate_x: posición de la bola con respecto a la placa central          Entrada (?)
    - plate_z: posición de la pelota con respecto al plato de home          Entrada (?)
    - pitch_mph: velocidad de la pelota lanzada                             Entrada
    - launch_speed: velocidad a la que la pelota del bate                   Entrada
    - launch_angle: angulo vertical de la pelota dejando el bate            Entrada
    - is_home_run: es home run                                              Salida
    - NAME: nombre del estadio                                              Entrada
    - Cover: indica si el estadio es abierto en la parte superior           Entrada
    - LF_Dim: distancia a la pared del jardin izquierdo                     Entrada
    - CF_Dim: distancia a la pared del campo central                        Entrada
    - RF_Dim: distancia a la pared del jardin derecho                       Entrada
    - LF_W: altura de la pared del jardin izquierdo                         Entrada
    - CF_W: altura de la pared del campo central                            Entrada
    - RF_W: altura de la pared del jardin derecho                           Entrada

#### Aclaraciones

- game_date: usa un formato (YYYY/MM/DD)
- is_batter_lefty: dos valores posibles:
    - 0: No
    - 1: Si
- is_pitcher_lefty: tiene dos valores posibles
    - 0: No
    - 1: Si
- bb_type:
    line_drive
    popup
    fly_ball
    ground_ball
- bearing: tiene tres posibles valores
    - left (izquierda)
    - center (centro)
    - rigth (derecha)
- plate_x: se indica con signo "-" si esta a la izquierda y con signo "+" si es hacia la derecha. Se mide en pies
- plate_z: se mide en pies
- pitch_mph: se mide en millas por hora
- launch_speed: se mide en millas por hora
- launch_angle: se expresan los grados relativos a una horizontal
- Cover: tiene tres valores posibles.
    - Dome (Cúpula)
    - Outdoor (Exterior)
    - Roof (Techo)
- LF_Dim: Se mide en pies
- CF_Dim: Se mide en pies
- RF_Dim: Se mide en pies
- LF_W: Se mide en pies
- CF_W: Se mide en pies
- RF_W: Se mide en pies
- Se va a trabajar con los archivos train y park_dimensions. Los mismos se van a unir mediante la variable "park"

### Detalle de variable de salida

La variable de salida es "is_home_run" que evalúa si una jugada termina en home run. Esto se produce si el bateador golpea la bola con fuerza y logra enviarla a las tribunas o fuera del estadio. Puede tener dos valores:
- 0: No es home run
- 1: Es home run

### Variables no utilizadas

- bip_id: No se usa una variable porque dicha información no es relevante para ser usada en el modelo y debido a que es un id que nunca se repite
- batter_id: No se usa porque se puede utilizar la variable nombre en su lugar, lo que sería más práctico porque brinda más información
- pitcher_id: No se usa porque se puede utilizar la variable nombre, lo que sería más práctico porqu brinda más información
- park: No se usa porque dicha información no representa mucho y en consecuencia se puede utilizar en su lugar el nombre del estadio. Hay que aclarar que dicha variable la vamos a usar para hacer un merge de los distintos archivos



## 2. Análisis detallado de un conjunto de variables

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import seaborn as sns

In [ ]:
entrenamiento = pd.read_csv('./train.csv')
estadio = pd.read_csv('./park_dimensions.csv')

In [ ]:
completa=entrenamiento.merge(estadio, on="park", how="left")

In [ ]:
renamed_columns = {'NAME': 'name', 'Cover': 'cover', 'LF_Dim': 'lf_dim', 'CF_Dim':'cf_dim',
                   'RF_Dim': 'rf_dim', 'LF_W': 'lf_w', 'CF_W': 'cf_w', 'RF_W': 'rf_w'
                  }
completa.rename(columns=renamed_columns, inplace=True)

In [ ]:
completa = completa.drop(['park','bip_id','batter_id','pitcher_id'],axis=1)

In [ ]:
completa.head()

In [ ]:
completa.describe()

### Para la variable de salida, explicar y graficar su balanceo y qué consecuencias va a tener eso a la hora de entrenar y medir el rendimiento de distintos modelos.

In [ ]:
completa.is_home_run.value_counts().plot.bar(title='Número de home runs');

In [ ]:
completa.is_home_run.value_counts(normalize=True).plot.bar(title='% de home_runs');

Luego de visualizar el gráfico, podemos ver que los valores de la salida no estan balanciados, ya que se tienen mas de 40000 (95% apróx) casos en los que no se tiene home run y solo menos de 5000 (5% apróx) donde si se tiene home run. Esta gran diferencia podría traer problemas a la hora de entrenar diferentes modelos, ya que estos podrían optar por responder siempre por el valor que aparece en la mayoría de los casos y no en dicho caso no tratarían de predecir.

### Para 5 variables de entrada (elegidas o no, pero incluyendo al menos 3 elegidas) graficar y explicar cómo se comportan y cómo afectan a la variable de salida.

Las 5 variables de entrada elegidas para dicho punto son:
- launch speed
- launch angle
- pitch_mph
- bearing
- name

Variable: launch_speed

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='launch_speed', color='is_home_run', barmode='group')

Como conclusión podemos decir que la variable de entrada afecta a la variable de salida ya que cuando alcanza valores entre las 90 y las 110 millas por hora aproximadamente se empiezan a ver ejemplos de home run. Dichos valores describe una distribución simétrica.

### Variable: launch_angle

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='launch_angle', color='is_home_run', barmode='group')

Como conclusión del gráfico anterior podemos ver que los datos de dicha variable de entrada tienen afectan a los da las variables de salida porque podemos ver un rango de valores entre los 18 y 42 grados aproximadamente en los que se produce home run. DIcho rango describe una distribución simétrica de datos.

### Variable: pitch_mph

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='pitch_mph', color='is_home_run', barmode='group', nbins=30)
px.histogram(completa, x='pitch_mph', barmode='group', nbins=30)

En este caso podemos ver que los datos de salida dependen de los datos de la variable de entrada pero en menor medida, ya que tenemos una distribución más uniforme con un rango de valores que va desde el 70 hasta los 100 millas por hora aproximadamente.

### Variable: bearing

In [ ]:
# Distribución de los datos de la variable "bearing"
completa.bearing.value_counts().plot.pie(autopct='%1.0f%%',figsize=(5,5))

En el gráfico anterior se puede ver que la mayor parte de los golpes son hacia el centro, y la segunda mayor cantidad es hacia la izquierda

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='bearing', color='is_home_run', barmode='group')

Como se puede ver en este caso, la variable tiene poca importancia en la salida, ya que teniendo en cuenta los tipos de lanzamiento y su desigual distribución, los casos en que se producen home runs en general son similares.

### Variable: name

In [ ]:
#Distribución de los datos
#completa.name.value_counts().plot.barh()
completa.name.value_counts().plot.pie(autopct='%1.0f%%',figsize=(10,10))

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='name', color='is_home_run', barmode='group')

Como se puede ver teniendo en cuenta la distribución y la relación con la variable de salida, podemos decir que la variable de entrada influye poco o casi nada en definir si una jugada es termina en home run o no.

### Para cada una de las variables de entrada elegidas, explicar si se debería realizar o no alguna transformación para poder utilizarla como entrada de un modelo y por qué.

In [ ]:
completa.sample(10)

Para utilizar el dataset en un modelo hay muchos cambios que se podrían aplicar a las variables. Los mismos se mencionan a continuación:

### Analizar si las variables de entrada seleccionadas presentan valores nulos y/o extremos. En caso de encontrar dichas condiciones, indicar qué tratamiento se podría darle a las mismas y por qué.

### Valores nulos

In [ ]:
completa.isnull().sum()

Como se puede ver en la estadistica anterior hay valores nulos en las variables "launch_speed" media, "launch_angle" y "bb_type" eliminar. El tratamiento que se le podría dar a las dos primeras es el de completarlas con la media de obtenida de los otros casos o con los valores de los casos similares. Otra opción podría ser eliminar sus filas ya que se tienen muchos datos y no habría muchos problemas sin se sacan algunos. Con respecto a la variable "bb_type" lo que se puede hacer es eliminar las filas con los valores nulos, ya que son muy pocas.

### Valores extremos

In [ ]:
# Convertimos en array a la columna launch_angle
launch_angles = np.array(completa.launch_angle)

# Armamos un diagrama de dispersión con la variable launch_angle.
angle_unique, counts = np.unique(launch_angles, return_counts = True)
size = counts * 3
colors = ['blue'] * len(angle_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(angle_unique,np.ones(len(angle_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
# DIagrama de caja de campo launch_angle

completa.boxplot('launch_angle');

In [ ]:
launch_speeds = np.array(completa.launch_speed)

speed_unique, counts = np.unique(launch_speeds, return_counts = True)
size = counts * 3
colors = ['blue'] * len(speed_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(speed_unique,np.ones(len(speed_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
completa.boxplot('launch_speed');

In [ ]:
plate_xs = np.array(completa.plate_x)

platex_unique, counts = np.unique(plate_xs, return_counts = True)
size = counts * 3
colors = ['blue'] * len(platex_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(platex_unique,np.ones(len(platex_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
completa.boxplot('plate_x');

In [ ]:
plate_zs = np.array(completa.plate_z)

platez_unique, counts = np.unique(plate_zs, return_counts = True)
size = counts * 3
colors = ['blue'] * len(platez_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(platez_unique,np.ones(len(platez_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
completa.boxplot('plate_z');

In [ ]:
pitch_mphs = np.array(completa.pitch_mph)

pitch_mph_unique, counts = np.unique(pitch_mphs, return_counts = True)
size = counts * 3
colors = ['blue'] * len(pitch_mph_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(pitch_mph_unique,np.ones(len(pitch_mph_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
completa.boxplot('pitch_mph');

De acuerdo a lo visto en los gráficos anteriores, las variables que poseen extremos son "plate_x", "plate_z", "pitch_mph", "launch_speed" y "launch_angle". Lo que se podría hacer con dichos datos es eliminarlos para que no afecten los resultados de los métricas.Cabe aclarar que las variables "lf_dim", "cf_dim", "rf_dim", "lf_w", "cf_w" y "rf_w" también presentan valores extremos, pero en estos casos se deben dejar porque hablan de las dimensiones de los estadios y por lo tanto no se deberían eliminar.

### Verificar si existen variables altamente correlacionadas con la variable "target". En dicho caso, explicar por qué considera que esto pasa. (hacer gráfico de colores)

In [ ]:
# Grafico de colores de las principales variables

sns.heatmap(completa.corr(), annot=True, cmap='RdYlGn', linewidths=0.2)
fig = plt.gcf()
fig.set_size_inches(20, 8)
plt.show()

Como se ve en el gráfico de colores no existen variables altamente correlacionadas con la variable target, ya que la variables con mayor valor de correlación son "launch_speed" y "launch_angle" con 0,25 y 0,13 respectivamente. Consideramos que esto pasa por la cantidad de variables de entrada del sataset y por el gran desbalance entre los datos de categoría positiva y negativa (si es o no home run).

## 3. Hipótesis sobre los datos

Formular y describir de forma coloquial al menos una hipótesis sobre los datos que podría explicar el valor de la variable "target" bajo determinadas condiciones. Realizar un análisis para contrastar si dicha hipótesis se valida o, por el contrario, se refuta a partir de los datos.

Describa y presente cualquier otro hallazgo que sea capaz de encontrar en los datos. Explique el impacto que podría tener en la creación de un modelo.